## Coupling the trained model with GCM outputs for future prediction

Here, we demonstrate how the trained model can be feed with simulated predictors to generate future projections. We only experiment with the RCP8.5 using the MPI-ESM CMIP5 datasets (please note that any GCN can be used if available)

In [6]:
# import all the models required
import os 
import sys 
import pandas as pd 
import numpy as np 
from collections import OrderedDict
import socket

# modules related to pyESD

from pyESD.Weatherstation import read_station_csv
from pyESD.standardizer import MonthlyStandardizer
from pyESD.ESD_utils import store_pickle, store_csv
from pyESD.splitter import KFold
from pyESD.ESD_utils import Dataset
from pyESD.Weatherstation import read_weatherstationnames

In [7]:
# define the predictors without teleconnection indices
predictors = ["t2m", "tp","msl", "v10", "u10", "NAO", "EA", "SCAN", "EAWR",
              "u250", "u850", "u500","u700", "u1000","v250", "v850", "v500","v700", "v1000",
              "r250", "r850", "r500","r700", "r1000", "z250", "z500", "z700", "z850", "z1000", 
              "t250", "t850", "t500","t700", "t1000","dtd250", "dtd850", "dtd500","dtd700", "dtd1000"
              ]

# date-range for model training and validation
from1958to2010 = pd.date_range(start="1958-01-01", end="2010-12-31", freq="MS")

# date-range for testing model
from2011to2020 = pd.date_range(start="2011-01-01", end="2020-12-31", freq="MS")

#full-time range
from1958to2020 = pd.date_range(start="1958-01-01", end="2020-12-31", freq="MS")

# date-range for the GCM output 
fullCMIP5 = pd.date_range(start='2010-01-01', end='2100-12-31', freq='MS')

# full amip date-range
fullAMIP = pd.date_range(start='1979-01-01', end='2000-12-31', freq='MS')

### control function

Define the control function that performs the predictor selection and model training.
1. read the station data as object that would apply all the ESD routines
2. set predictors with the list of predictors defined and the radius to construct the regional means
3. standardize the data with any of the standardizers. Here we use the MonthlyStandardizer method
4. defined the scoring metrics to be used for the validation
5. set the model to be used for the ESD training (here we will use the Stacking of the good models from the prevous)
6. fit the model, here we have to define the predictor selector method (here: Recursive ) to be used for selecting the predictors
7. get the selected predictors 
8. use the cross_validate_predict to get the cross-validation metrics of the model training 
9. future predictions using RCP85
10. stored the validation metrics

In [8]:
# import the variables required for the control script
from read_data import radius, station_prec_datadir, stationnames_prec, ERA5Data, cachedir_prec
# import the GCM data (here CMIP5 MPI-ESM RCP85)
from read_data import CMIP5_RCP85_R1, CMIP5_AMIP_R1

In [9]:
def run_final_model(variable, cachedir, 
                    stationnames, station_datadir, method, 
                    final_estimator=None,
                    base_estimators=None, ensemble_learning=False):

    num_of_stations = len(stationnames)
    
    
    
        
    for i in range(num_of_stations):
        
        stationname = stationnames[i]
        station_dir = os.path.join(station_datadir, stationname + ".csv")
        SO = read_station_csv(filename=station_dir, varname=variable)
        
        
        # USING ERA5 DATA
        # ================
        
        
        #setting predictors 
        SO.set_predictors(variable, predictors, predictordir, radius,)
        
        #setting standardardizer
        SO.set_standardizer(variable, standardizer=MonthlyStandardizer(detrending=False,
                                                                        scaling=False))
        
        scoring = ["neg_root_mean_squared_error",
                   "r2", "neg_mean_absolute_error"]
        
        #setting model
        SO.set_model(variable, method=method, ensemble_learning=ensemble_learning, 
                      estimators=base_estimators, final_estimator_name=final_estimator, 
                      daterange=from1958to2010,
                      predictor_dataset=ERA5Data, 
                      cv=KFold(n_splits=10),
                      scoring = scoring)
        
        
        # MODEL TRAINING (1958-2000)
        # ==========================
        
        
        SO.fit(variable, from1958to2010, ERA5Data, fit_predictors=True, predictor_selector=True, 
                    selector_method="Recursive" , selector_regressor="ARD",
                    cal_relative_importance=False)
            
        score_1958to2010, ypred_1958to2010 = SO.cross_validate_and_predict(variable, from1958to2010, ERA5Data)
            
        score_2011to2020 = SO.evaluate(variable, from2011to2020, ERA5Data)
        
        ypred_1958to2010 = SO.predict(variable, from1958to2010, ERA5Data)
            
        ypred_2011to2020 = SO.predict(variable, from2011to2020, ERA5Data)
        
        # STORING SCORES
        # ==============
            
        store_pickle(stationname, "validation_score_" + ensemble_method, score_1958to2010, cachedir)
        store_pickle(stationname, "test_score_" + ensemble_method, score_2011to2020, cachedir)
        
        #USING CMIP5 DATA + MODEL 1958-2000
        #====================================
        
        print("fitting the AMIP predictors based on the selected model---all from realisation 1")
        SO.fit_predictor(variable, predictors, fullAMIP, CMIP5_AMIP_R1) 
        
        print("predicting based on the RCP 8.5 predictors")
        yhat_CMIP5_RCP85_R1_anomalies = SO.predict(variable, fullCMIP5, 
                                        CMIP5_RCP85_R1, fit_predictors=True, fit_predictand=True,
                                        params_from="CMIP5_AMIP_R1", patterns_from= "CMIP5_AMIP_R1")
        
        
        
        # PREDICTING THE ABSOLUTE VALUES
        # ===============================
        
        
        print("predicting based on the RCP 8.5 predictors")
        yhat_CMIP5_RCP85_R1 = SO.predict(variable, fullCMIP5, 
                                        CMIP5_RCP85_R1, fit_predictors=True, fit_predictand=False,
                                        params_from="CMIP5_AMIP_R1", patterns_from= "CMIP5_AMIP_R1")
        
        
        # STORING OF RESULTS
        # ===================
        
        y_obs_1958to2010 = SO.get_var(variable, from1958to2010, anomalies=True)
            
        y_obs_2011to2020 = SO.get_var(variable, from2011to2020, anomalies=True)
            
        y_obs_1958to2020 = SO.get_var(variable, from1958to2020, anomalies=True)
        
        y_obs_1958to2020_true = SO.get_var(variable, from1958to2020, anomalies=False)
        
        
        predictions = pd.DataFrame({
            "obs": y_obs_1958to2020_true,
            "obs anomalies": y_obs_1958to2020,
            "obs 1958-2010": y_obs_1958to2010,
            "obs 2011-2020": y_obs_2011to2020,
            "ERA5 1958-2010": ypred_1958to2010,
            "ERA5 2011-2020": ypred_2011to2020,
            "CMIP5 RCP8.5 anomalies":yhat_CMIP5_RCP85_R1_anomalies,
            "CMIP5 RCP8.5":yhat_CMIP5_RCP85_R1,
            })
        
        store_csv(stationname, "predictions_" + ensemble_method, predictions, cachedir)

In [10]:
ensemble_method = "Stacking"
 
base_estimators = ["LassoLarsCV", "ARD", "RandomForest", "Bagging"]

final_estimator = "ExtraTree"

In [11]:
run_final_model(variable="Precipitation", cachedir=cachedir_prec, 
                    stationnames=stationnames_prec, station_datadir=station_prec_datadir, 
                    method=ensemble_method, 
                    final_estimator=final_estimator,
                    base_estimators=base_estimators, ensemble_learning=True)

Freiburg 48.0232 7.8343 236.0
-------using ExtraTree as final estimator ......
13 : optimal number of predictors and selected variables are Index(['t2m', 'tp', 'v10', 'u10', 'NAO', 'SCAN', 'u850', 'v1000', 'r1000',
       't500', 'dtd250', 'dtd700', 'dtd1000'],
      dtype='object')
RMSE: 26.465655
Nach-Sutcliffe Efficiency(NSE): 0.485140
Mean Squared Error): 26.47
Mean Absolute Error): 19.35
Explained Variance: 0.53
R² (Coefficient of determinaiton): 0.49
Maximum error: 83.888169
Adjusted R²: 0.56
fitting the AMIP predictors based on the selected model---all from realisation 1
Regenerating predictor data for t2m using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for tp using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for msl using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating p

Regenerating predictor data for u1000 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v250 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v850 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v500 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v700 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v1000 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for r250 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for 

Regenerating predictor data for dtd250 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for dtd850 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for dtd500 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for dtd700 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for dtd1000 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
predicting based on the RCP 8.5 predictors
Regenerating predictor data for t2m using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for tp using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R

Regenerating predictor data for v1000 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for r250 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for r850 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for r500 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for r700 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for r1000 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for z250 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for z500 using dataset CM

fitting the AMIP predictors based on the selected model---all from realisation 1
Regenerating predictor data for t2m using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for tp using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for msl using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for v10 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for u10 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for u250 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R1 and CMIP5_AMIP_R1
Regenerating predictor data for u850 using dataset CMIP5_AMIP_R1 with loading patterns and params from CMIP5_AMIP_R

Regenerating predictor data for t250 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for t850 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for t500 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for t700 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for t1000 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for dtd250 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for dtd850 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data f

Regenerating predictor data for v850 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v500 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v700 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for v1000 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for r250 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for r850 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for r500 using dataset CMIP5_RCP85_R1 with loading patterns and params from CMIP5_RCP85_R1 and CMIP5_RCP85_R1
Regenerating predictor data for r